In [1]:
#!/usr/bin/env python
# coding: utf-8

import os, sys
import numpy as np

import xarray as xr
import pandas as pd

from dask.distributed import Client, LocalCluster
from dask.diagnostics import ProgressBar
#
# Initialisation d'un cluster de 32 coeurs
cluster = LocalCluster(processes=False, n_workers=1, threads_per_worker=8)
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 12,Total memory: 78.61 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:37445,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 12
Started: Just now,Total memory: 78.61 GiB
Comm: tcp://127.0.0.1:45965,Total threads: 3
Dashboard: http://127.0.0.1:42045/status,Memory: 19.65 GiB
Nanny: tcp://127.0.0.1:45741,


In [2]:
indir_clim = '/cnrm/tropics/commun/DATACOMMUN/WAVE/NO_SAVE/DATA/SMOTHED_CLIM/'
indir_data = '/cnrm/tropics/commun/DATACOMMUN/ERA5/0.25/netcdf/sfc_3h/'

var = 'tcwv'

ds_data = xr.open_mfdataset(indir_data+'*'+var+'*.nc', chunks = {'time' : 1}, parallel=True)
ds_clim = xr.open_dataset(indir_clim+'clim_tcwv_smooth_ERA5_3H_1990_2020.nc', chunks = {'time' : 1})

outdir = '/cnrm/tropics/commun/DATACOMMUN/WAVE/NO_SAVE/DATA/RAW_ANOMALY/'

In [3]:
def hour_mean(x):
     return x.groupby('time.hour').mean('time')
    
def hour_sum(x):
     return x.groupby('time.hour').sum('time')
    
    
def isLeapYear (yearN):
    if ((yearN % 4 == 0) and (yearN % 100 != 0)) or (yearN % 400 == 0):
        reponse = True
    else:
        reponse = False
    print(reponse, '\n')
    return reponse

In [9]:
year = np.arange(1990,2021)
for y in year :
    print(y)
    ds = ds_data.sel(time = str(y))
    if isLeapYear(ds.time.dt.year.values[0]) == False :
        _ds_clim = ds_clim.isel(time = slice(0,365*int(24/3)))
        _ds_clim['time'] = ds['time']
    else :
        _ds_clim = ds_clim
        _ds_clim['time'] = ds['time']


    da = ds['tcwv'] - _ds_clim['tcwv_smooth']
    da = da.to_dataset(name = "tcwv_ano")
    da.to_netcdf(outdir + 'anom_tcwv_brut_ERA5_3H_'+str(y)+'.nc', compute = False)
    print(da)

    with ProgressBar():
        da = da.compute()


1990
False 

<xarray.Dataset>
Dimensions:    (longitude: 1440, latitude: 721, time: 2920)
Coordinates:
  * longitude  (longitude) float32 0.0 0.25 0.5 0.75 ... 359.0 359.2 359.5 359.8
  * latitude   (latitude) float32 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * time       (time) datetime64[ns] 1990-01-01 ... 1990-12-31T21:00:00
Data variables:
    tcwv_ano   (time, latitude, longitude) float64 dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>


KeyboardInterrupt: 